In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

# 커널을 구성하다보면 에러는 아니지만, 빨간색 네모 박스 warning이 뜨는 경우가 많다. 
# 그런 보기 싫은 부분들을 제거해주겠다.
# import warnings
# warnings.filterwarnings('ignore')

# os 패키지를 통해 현재 디렉토리 위치를 변경하고, read_csv를 더 편리하게 할 수 있음
import os
os.getcwd() # 현재 디렉토리 파악
# os.chdir(r"______") # 불러오고 싶은 파일이 위치한 주소를 ___에 입력

# 다른 노트북 작성할 때도 이 셀만 떼서 사용 가능하다.
import platform                

# 웬만하면 해주는 것이 좋다
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin': #맥os 사용자의 경우에
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic')
    
elif platform.system() == 'Windows':#윈도우 사용자의 경우에
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.style.use('seaborn-v0_8-whitegrid') # https://python-graph-gallery.com/199-matplotlib-style-sheets/
    rc('font', family=font_name)
    
# tqdm
from tqdm.notebook import tqdm

In [16]:
path = 'c:/reposit/data/dacon/2023 전력사용량 예측 AI 경진대회/'

import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action='ignore') 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 데이터 로드
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
building_df = pd.read_csv(path + 'building_info.csv')

# building info 전처리
building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].str.replace('-', '0')
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].str.replace('-', '0')
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].str.replace('-', '0')

building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].astype(float)
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].astype(float)
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].astype(float)


# 조인
train_df = pd.merge(train_df, building_df, on='건물번호', how='left')
test_df = pd.merge(test_df, building_df, on='건물번호', how='left')

#결측값을 0으로 채웁니다
train_df = train_df.fillna(0)

# 날짜 타입으로 변경
train_df['일시'] = pd.to_datetime(train_df['일시'])

#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].dt.month
train_df['day'] = train_df['일시'].dt.day
train_df['time'] = train_df['일시'].dt.hour
train_df['weekday'] = train_df['일시'].dt.weekday

train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형'])
train_y = train_df['전력소비량(kWh)']

model = RandomForestRegressor()
model.fit(train_x, train_y)

# 날짜 타입으로 변경
test_df['일시'] = pd.to_datetime(test_df['일시'])

test_df['month'] = test_df['일시'].dt.month
test_df['day'] = test_df['일시'].dt.day
test_df['time'] = test_df['일시'].dt.hour
test_df['weekday'] = test_df['일시'].dt.weekday

test_x = test_df.drop(columns=['num_date_time', '일시', '건물유형'])

preds = model.predict(test_x)

submission = pd.read_csv(path + 'sample_submission.csv')
submission['answer'] = preds

submission.to_csv(path + 'submission_5.csv', index=False)

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204000 entries, 0 to 203999
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   num_date_time  204000 non-null  object        
 1   건물번호           204000 non-null  int64         
 2   일시             204000 non-null  datetime64[ns]
 3   기온(C)          204000 non-null  float64       
 4   강수량(mm)        204000 non-null  float64       
 5   풍속(m/s)        204000 non-null  float64       
 6   습도(%)          204000 non-null  float64       
 7   일조(hr)         204000 non-null  float64       
 8   일사(MJ/m2)      204000 non-null  float64       
 9   전력소비량(kWh)     204000 non-null  float64       
 10  건물유형           204000 non-null  object        
 11  연면적(m2)        204000 non-null  float64       
 12  냉방면적(m2)       204000 non-null  float64       
 13  태양광용량(kW)      204000 non-null  float64       
 14  ESS저장용량(kWh)   204000 non-null  float64       
 15  